# 🧪 Lab Supply Calculator

Welcome! Use this tool to:

- Select a protocol
- Enter your sample size
- Click "Run All"
- Receive a downloadable CSV with supply needs and reorder status

🔒 **Note**: You'll need a GitHub access token to access the files. Please see Aubrie or Vicki for this.

---

## 🧭 Instructions

1. Enter your **GitHub token**
2. Enter the **sample size**
3. Choose the **protocol**
4. Click **Runtime → Run All**
5. Download the CSV report when prompted

---

### Example:
- Token: `ghp_XXXXXXXXXXXXXXXXXXXX`
- Sample size: `50`


In [22]:
#@title Install packages and imports


##Install packages and imports
# Installing packages
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import yaml
except ImportError:
    install_package("pyyaml")
    import yaml

# Import all required libraries
import requests
import csv
import pandas as pd
import os
from io import StringIO
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

print("✅ All packages installed and imported successfully!")

# Define GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
INVENTORY_PATH = "test_protocol_check/inventory/inventory.csv"
PROTOCOL_DIR = "test_protocol_check/protocols"

print("✅ Configuration set")

#Helper Functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path, token):
    url = github_raw_url(path)
    headers = {"Authorization": f"token {token}"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {path}: {e}")
        raise

def get_protocol_list(token, user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    headers = {"Authorization": f"token {token}"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        files = r.json()
        return [f['path'] for f in files if f['name'].endswith('.yaml')]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching protocol list: {e}")
        return []

def load_inventory(text):
    inv = {}
    try:
        # Remove BOM if present
        text = text.lstrip('\ufeff')
        reader = csv.DictReader(StringIO(text))
        for row in reader:
            item = row['Item'].strip()
            inv[item] = {
                'unit': row['Unit'].strip(),
                'stock': float(row['Stock Quantity']),
                'threshold': float(row['Reorder Threshold'])
            }
        return inv
    except Exception as e:
        print(f"❌ Error loading inventory: {e}")
        return {}

def load_protocol(text):
    try:
        return yaml.safe_load(text)
    except Exception as e:
        print(f"❌ Error loading protocol: {e}")
        return {}

def calculate_needs(protocol_data, sample_count):
    if 'supplies_per_sample' not in protocol_data:
        print("❌ Protocol missing 'supplies_per_sample' section")
        return {}
    return {item: qty * sample_count for item, qty in protocol_data['supplies_per_sample'].items()}

print("✅ Helper functions defined")

✅ All packages installed and imported successfully!
✅ Configuration set
✅ Helper functions defined


In [23]:
#@title User Input Interface
print("📝 Please enter your details below:")

# Create input widgets
token_input = widgets.Text(
    value='',
    placeholder='Enter your GitHub token here (ghp_...)',
    description='GitHub token:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)

sample_input = widgets.IntText(
    value=50,
    description='Sample size:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Create a button to proceed
proceed_button = widgets.Button(
    description='Continue with these settings',
    disabled=False,
    button_style='success',
    tooltip='Click to proceed with the entered token and sample size',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

# Status output
status_output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with status_output:
        status_output.clear_output()
        if not token_input.value or len(token_input.value) < 10:
            print("❌ Please enter a valid GitHub token")
            return
        if sample_input.value <= 0:
            print("❌ Please enter a valid sample size")
            return
        print(f"✅ Token received: {token_input.value[:4]}...")
        print(f"✅ Sample size: {sample_input.value}")
        print("✅ Ready to proceed to next step!")
        proceed_button.disabled = True
        proceed_button.description = "Settings confirmed ✓"

proceed_button.on_click(on_button_click)

# Display all widgets
display(token_input, sample_input, proceed_button, status_output)

print("👆 Enter your GitHub token and sample size above, then click 'Continue' to proceed")


📝 Please enter your details below:


Text(value='', description='GitHub token:', layout=Layout(width='80%'), placeholder='Enter your GitHub token h…

IntText(value=50, description='Sample size:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Continue with these settings', layout=Layout(margin='10px 0px', wi…

Output()

👆 Enter your GitHub token and sample size above, then click 'Continue' to proceed


In [24]:
#@title Fetch Protocols and Create Dropdown
# Note: Run this cell AFTER clicking "Continue" in the previous cell

print("🔍 Fetching available protocols...")

# Check if token is entered
if not token_input.value or len(token_input.value) < 10:
    print("❌ Please enter a valid GitHub token in the previous cell first!")
    print("⬆️ Go back to the previous cell and enter your token")
else:
    try:
        protocols = get_protocol_list(
            token_input.value,
            GITHUB_USER,
            REPO_NAME,
            PROTOCOL_DIR,
            BRANCH
        )

        if not protocols:
            print("❌ No protocols found or error accessing repository")
            protocols = [
                "test_protocol_check/protocols/dna_extraction_mn.yaml",
                "test_protocol_check/protocols/pcr_setup.yaml"
            ]
            print("📋 Using default protocol list")
        else:
            print(f"✅ Found {len(protocols)} protocols")

        # Create protocol dropdown
        protocol_dropdown = widgets.Dropdown(
            options=protocols,
            description='Protocol:',
            layout=widgets.Layout(width='95%'),
            style={'description_width': 'initial'}
        )

        display(protocol_dropdown)
        print("✅ Protocol dropdown ready!")

    except Exception as e:
        print(f"❌ Error fetching protocols: {e}")
        print("📋 Using default protocol list")
        protocols = [
            "test_protocol_check/protocols/dna_extraction_mn.yaml",
            "test_protocol_check/protocols/pcr_setup.yaml"
        ]
        protocol_dropdown = widgets.Dropdown(
            options=protocols,
            description='Protocol:',
            layout=widgets.Layout(width='95%'),
            style={'description_width': 'initial'}
        )
        display(protocol_dropdown)
        print("✅ Default protocol dropdown ready!")



🔍 Fetching available protocols...
❌ Please enter a valid GitHub token in the previous cell first!
⬆️ Go back to the previous cell and enter your token


In [25]:
#@title Process Data and Generate Report
# Note: Run this cell AFTER selecting your protocol in the previous cell

print("\n🔄 Processing data...")

# Check if all required inputs are available
if not token_input.value or len(token_input.value) < 10:
    print("❌ Please enter a valid GitHub token first!")
    print("⬆️ Go back to Cell 5 and enter your token")
elif 'protocol_dropdown' not in locals():
    print("❌ Protocol dropdown not found!")
    print("⬆️ Please run Cell 6 first to load protocols")
else:
    try:
        # Fetch inventory and protocol files
        print("📥 Fetching inventory data...")
        inventory_text = fetch_file(INVENTORY_PATH, token_input.value)

        print("📥 Fetching protocol data...")
        protocol_text = fetch_file(protocol_dropdown.value, token_input.value)

        # Parse data
        print("🔍 Parsing data...")
        inventory = load_inventory(inventory_text)
        protocol = load_protocol(protocol_text)

        if not inventory:
            raise Exception("Failed to load inventory data")
        if not protocol:
            raise Exception("Failed to load protocol data")

        # Calculate needs
        needs = calculate_needs(protocol, sample_input.value)

        if not needs:
            raise Exception("Failed to calculate supply needs")

        print(f"✅ Calculated needs for {len(needs)} items")

        # Create report
        report_rows = []
        per_sample = protocol.get("supplies_per_sample", {})

        for item, total_required in needs.items():
            per_unit = per_sample.get(item, "N/A")
            inv = inventory.get(item)

            if not inv:
                report_rows.append({
                    "Item": item,
                    "Per Sample": per_unit,
                    "Need": total_required,
                    "Stock": "N/A",
                    "Status": "MISSING",
                    "Reorder": "⚠️"
                })
            else:
                stock = inv['stock']
                threshold = inv['threshold']
                status = "OK" if stock >= total_required else "LOW"
                reorder_flag = "YES" if (stock - total_required) < threshold else "NO"
                report_rows.append({
                    "Item": item,
                    "Per Sample": per_unit,
                    "Need": round(total_required, 2),
                    "Stock": round(stock, 2),
                    "Status": status,
                    "Reorder": reorder_flag
                })

        # Convert to DataFrame
        df = pd.DataFrame(report_rows)

        # Display report
        print(f"\n📋 Supply Report for {sample_input.value} samples using {os.path.basename(protocol_dropdown.value)}")
        print("=" * 80)
        display(df)

        print("✅ Report generated successfully!")

    except Exception as e:
        print(f"❌ Error processing data: {e}")
        df = pd.DataFrame()



🔄 Processing data...
❌ Please enter a valid GitHub token first!
⬆️ Go back to Cell 5 and enter your token


In [26]:
#@title Save and Download Report
if not df.empty:
    try:
        print("\n💾 Preparing CSV download...")
        # Extract protocol name
        protocol_name = os.path.basename(protocol_dropdown.value)

        # Create metadata
        metadata_info = [
            ["Protocol:", protocol_name],
            ["Sample Size:", sample_input.value],
            ["Generated:", pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")],
            [""], # Blank row
        ]

        # Save to CSV with metadata first
        output_path = "/content/supply_report.csv"

        # Write metadata first
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            # Write metadata rows
            for row in metadata_info:
                writer.writerow(row)

        # Append the DataFrame with headers
        df.to_csv(output_path, mode='a', index=False)

        print("✅ CSV report generated successfully!")
        print("📁 Downloading file...")

        # Download the file in Google Colab
        from google.colab import files
        files.download(output_path)

    except Exception as e:
        print(f"❌ Error generating report: {str(e)}")
else:
    print("❌ No data to export")


💾 Preparing CSV download...
✅ CSV report generated successfully!
📁 Downloading file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>